# Import and drop redundant columns

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sqlalchemy import create_engine
from scipy.stats import zscore
import matplotlib.pyplot as plt
import joblib
import logging
from datetime import datetime
logging.basicConfig(level=logging.INFO)


In [2]:
class SalesPerformancePrediction:
    def __init__(self):
        self.label_encoders = {}
        self.models = {}
        self.scaler = StandardScaler()

    def fetch_data_from_sql(self, query, server, database):
        """
        Fetch data from SQL Server using SQLAlchemy
        """
        logging.info("Fetching data from SQL Server...")
        engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")
        data = pd.read_sql_query(query, engine)
        return data

    def prepare_data(self, fact_sales, dim_product, dim_date, dim_territory):
        """
        Merge dimension tables with fact_sales and calculate metrics
        """
        logging.info("Merging dimension tables with fact_sales...")
        sales_data = fact_sales.merge(dim_product, on='ProductKey')\
                            .merge(dim_date, on='DateKey')\
                            .merge(dim_territory, on='TerritoryKey')


        # Handle missing and invalid values
        logging.info("Handling missing and invalid values...")
        sales_data.replace([np.inf, -np.inf], np.nan, inplace=True)
        sales_data.fillna(0, inplace=True)


        # Remove outliers
        logging.info("Removing outliers...")
        sales_data['Revenue'] = sales_data['OrderQty'] * sales_data['UnitPrice']
        sales_data = sales_data[(np.abs(zscore(sales_data['Revenue'])) < 3)]
        sales_data = sales_data.drop(['Revenue'], axis=1)

        return sales_data

In [3]:
# Database Connection Details
server = "DESKTOP-TGOL65T"
database = "CompanyXdwh_real"

# Queries
fact_sales_query = "SELECT * FROM fact_sales"
dim_product_query = "SELECT * FROM dim_product"
dim_date_query = "SELECT * FROM dim_date"
dim_territory_query = "SELECT * FROM dim_territory"

# Initialize Class
prediction = SalesPerformancePrediction()

# Fetch Data
fact_sales = prediction.fetch_data_from_sql(fact_sales_query, server, database)
dim_product = prediction.fetch_data_from_sql(dim_product_query, server, database)
dim_date = prediction.fetch_data_from_sql(dim_date_query, server, database)
dim_territory = prediction.fetch_data_from_sql(dim_territory_query, server, database)

# Prepare Data
sales_data = prediction.prepare_data(fact_sales, dim_product, dim_date, dim_territory)


INFO:root:Fetching data from SQL Server...
INFO:root:Fetching data from SQL Server...
INFO:root:Fetching data from SQL Server...
INFO:root:Fetching data from SQL Server...
INFO:root:Merging dimension tables with fact_sales...
INFO:root:Handling missing and invalid values...
INFO:root:Removing outliers...


In [4]:
print(sales_data[sales_data.duplicated()])

Empty DataFrame
Columns: [SalesKey, SalesOrderID, ProductKey, DateKey, TerritoryKey, StoreKey, CustomerKey, OrderQty, UnitPrice, LineTotal, TotalDue, ModifiedDate_x, ProductID, ProductName, Category, Subcategory, Model, ListPrice, StandardCost, ModifiedDate_y, Day, Week, Month, Quarter, Year, TerritoryID, Name, Country, Group, SalesYTD, SalesLastYear, ModifiedDate]
Index: []

[0 rows x 32 columns]


In [5]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118634 entries, 0 to 121241
Data columns (total 32 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   SalesKey        118634 non-null  int64         
 1   SalesOrderID    118634 non-null  object        
 2   ProductKey      118634 non-null  int64         
 3   DateKey         118634 non-null  int64         
 4   TerritoryKey    118634 non-null  int64         
 5   StoreKey        118634 non-null  float64       
 6   CustomerKey     118634 non-null  float64       
 7   OrderQty        118634 non-null  int64         
 8   UnitPrice       118634 non-null  float64       
 9   LineTotal       118634 non-null  float64       
 10  TotalDue        118634 non-null  float64       
 11  ModifiedDate_x  118634 non-null  datetime64[ns]
 12  ProductID       118634 non-null  int64         
 13  ProductName     118634 non-null  object        
 14  Category        118634 non-null  object  

In [6]:
sales_data = sales_data.drop(['SalesKey', 'ProductKey', 'DateKey', 'TerritoryKey',
                             'StoreKey', 'CustomerKey', 'ProductID',
                             'ModifiedDate_x', 'ModifiedDate_y', 'ModifiedDate'],axis=1)
print(sales_data[sales_data.duplicated()])

Empty DataFrame
Columns: [SalesOrderID, OrderQty, UnitPrice, LineTotal, TotalDue, ProductName, Category, Subcategory, Model, ListPrice, StandardCost, Day, Week, Month, Quarter, Year, TerritoryID, Name, Country, Group, SalesYTD, SalesLastYear]
Index: []

[0 rows x 22 columns]


In [7]:
sales_data.columns

Index(['SalesOrderID', 'OrderQty', 'UnitPrice', 'LineTotal', 'TotalDue',
       'ProductName', 'Category', 'Subcategory', 'Model', 'ListPrice',
       'StandardCost', 'Day', 'Week', 'Month', 'Quarter', 'Year',
       'TerritoryID', 'Name', 'Country', 'Group', 'SalesYTD', 'SalesLastYear'],
      dtype='object')

In [8]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118634 entries, 0 to 121241
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   SalesOrderID   118634 non-null  object 
 1   OrderQty       118634 non-null  int64  
 2   UnitPrice      118634 non-null  float64
 3   LineTotal      118634 non-null  float64
 4   TotalDue       118634 non-null  float64
 5   ProductName    118634 non-null  object 
 6   Category       118634 non-null  object 
 7   Subcategory    118634 non-null  object 
 8   Model          118634 non-null  object 
 9   ListPrice      118634 non-null  float64
 10  StandardCost   118634 non-null  float64
 11  Day            118634 non-null  int64  
 12  Week           118634 non-null  int64  
 13  Month          118634 non-null  int64  
 14  Quarter        118634 non-null  int64  
 15  Year           118634 non-null  int64  
 16  TerritoryID    118634 non-null  int64  
 17  Name           118634 non-null  ob

In [9]:
sales_data.head()

,SalesOrderID,OrderQty,UnitPrice,LineTotal,TotalDue,ProductName,Category,Subcategory,Model,ListPrice,...,Week,Month,Quarter,Year,TerritoryID,Name,Country,Group,SalesYTD,SalesLastYear
0,43697-353,1,3578.2700,3578.2700,3953.9884,"Road-150 Red, 62",Bikes,Road Bikes,Road-150,3578.27,...,23,5,2,2011,6,Canada,Canada,North America,6.771829e+06,5.693989e+06
1,43698-354,1,3399.9900,3399.9900,3756.9890,"Mountain-100 Silver, 44",Bikes,Mountain Bikes,Mountain-100,3399.99,...,23,5,2,2011,7,France,France,Europe,4.772398e+06,2.396540e+06
2,43699-355,1,3399.9900,3399.9900,3756.9890,"Mountain-100 Silver, 44",Bikes,Mountain Bikes,Mountain-100,3399.99,...,23,5,2,2011,1,Northwest,United States,North America,7.887187e+06,3.298694e+06
3,43700-356,1,699.0982,699.0982,772.5036,"Road-650 Black, 62",Bikes,Road Bikes,Road-650,782.99,...,23,5,2,2011,4,Southwest,United States,North America,1.051085e+07,5.366576e+06
4,43701-357,1,3399.9900,3399.9900,3756.9890,"Mountain-100 Silver, 44",Bikes,Mountain Bikes,Mountain-100,3399.99,...,23,5,2,2011,9,Australia,Australia,Pacific,5.977815e+06,2.278549e+06


#  Feature Engineering

In [65]:
class FeatureEngineering:
    def __init__(self, df):
        self.df = df
    
    
    def derive_week(day, month, year):
        """
        Derives the ISO week number from day, month, and year.

        Parameters:
            day (int): The day of the month (1-31).
            month (int): The month of the year (1-12).
            year (int): The year (e.g., 2023).

        Returns:
            int: The ISO week number (1-52 or 1-53 depending on the year).
        """
        try:
            date = datetime(year, month, day)
            week = date.isocalendar().week
            return week
        except ValueError as e:
            print(f"Invalid date: {e}")
            return None
    
        
    def add_is_weekend_features(self):
        """
        Add date-related features like Day of Week, Is Weekend, and Month Name.
        """
        self.df['Date'] = pd.to_datetime(self.df[['Year', 'Month', 'Day']])
        self.df['Day_of_Week'] = self.df['Date'].dt.day_name()
        self.df['Is_Weekend'] = self.df['Day_of_Week'].isin(['Saturday', 'Sunday']).astype(int)
        self.df = self.df.drop(['Date', 'Day_of_Week'], axis=1)
        return self
    
    
    def add_cyclical_features(self):
        """
        Add cyclical features for month and day using sine and cosine transformations.
        """
        self.df['Month_Sin'] = np.sin(2 * np.pi * self.df['Month'] / 12)
        self.df['Month_Cos'] = np.sqrt(1 - self.df['Month_Sin'])
        self.df['Day_Sin'] = np.sin(2 * np.pi * self.df['Day'] / 31)
        self.df['Day_Cos'] = np.sqrt(1 - self.df['Day_Sin'])
        self.df = self.df.drop(['Month', 'Day'], axis=1)
        return self
    
    
    def add_ohe_category(self):
        """
        Add one-hot encoded features for the 'Category' column and drop the original column.
        """
        # Initialize the OneHotEncoder
        encoder = OneHotEncoder()

        # Fit and transform the 'Category' column
        encoded_categories = encoder.fit_transform(self.df[['Category']]).toarray()
        
        # Create a DataFrame from the encoded array with correct column names
        encoded_df = pd.DataFrame(encoded_categories, 
                                  columns=encoder.get_feature_names_out(),
                                  index=self.df.index)
        self.df = pd.concat([self.df, encoded_df], axis=1)
        self.df = self.df.drop(['Category'], axis=1)
        
        return self
    
    
    def process_SalesOrderID_column(self):
        # Assuming 'SalesOrderID' is a column in your DataFrame
        self.df[['OrderID', 'ProductID']] = self.df['SalesOrderID'].str.split('-', expand=True)

        # Convert to integers if needed
        self.df['OrderID'] = self.df['OrderID'].astype(int)
        self.df['ProductID'] = self.df['ProductID'].astype(int)

        # Drop the original 'SalesOrderID' column if no longer needed
        self.df = self.df.drop(columns=['SalesOrderID', 'ProductID'], axis=1)

        
        return self
    
    
    def get_data(self):
        """
        Return the engineered DataFrame.
        """
        return self.df
    
    


In [66]:
# drop columns Week and Quarter because its can be deduced from day, month and year
df = sales_data.drop(['Week', 'Quarter', 'Model', 'ProductName', 'Subcategory', 'Name', 'Country', 'Group'],axis=1)
print(df[df.duplicated()])

Empty DataFrame
Columns: [SalesOrderID, OrderQty, UnitPrice, LineTotal, TotalDue, Category, ListPrice, StandardCost, Day, Month, Year, TerritoryID, SalesYTD, SalesLastYear]
Index: []


In [69]:
fe = FeatureEngineering(df)
fe.add_is_weekend_features()
fe.add_cyclical_features()
fe.add_ohe_category()
fe.process_SalesOrderID_column()
transformed_df = fe.get_data()
transformed_df.head()

,OrderQty,UnitPrice,LineTotal,TotalDue,ListPrice,StandardCost,Year,TerritoryID,SalesYTD,SalesLastYear,Is_Weekend,Month_Sin,Month_Cos,Day_Sin,Day_Cos,Category_Accessories,Category_Bikes,Category_Clothing,Category_Components,OrderID
0,1,3578.2700,3578.2700,3953.9884,3578.27,2171.2942,2011,6,6.771829e+06,5.693989e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43697
1,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,7,4.772398e+06,2.396540e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43698
2,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,1,7.887187e+06,3.298694e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43699
3,1,699.0982,699.0982,772.5036,782.99,486.7066,2011,4,1.051085e+07,5.366576e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43700
4,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,9,5.977815e+06,2.278549e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43701


In [70]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118634 entries, 0 to 121241
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   OrderQty              118634 non-null  int64  
 1   UnitPrice             118634 non-null  float64
 2   LineTotal             118634 non-null  float64
 3   TotalDue              118634 non-null  float64
 4   ListPrice             118634 non-null  float64
 5   StandardCost          118634 non-null  float64
 6   Year                  118634 non-null  int64  
 7   TerritoryID           118634 non-null  int64  
 8   SalesYTD              118634 non-null  float64
 9   SalesLastYear         118634 non-null  float64
 10  Is_Weekend            118634 non-null  int32  
 11  Month_Sin             118634 non-null  float64
 12  Month_Cos             118634 non-null  float64
 13  Day_Sin               118634 non-null  float64
 14  Day_Cos               118634 non-null  float64
 15  Categ

In [68]:
transformed_df.isnull().sum()

OrderQty                0
UnitPrice               0
LineTotal               0
TotalDue                0
ListPrice               0
StandardCost            0
Year                    0
TerritoryID             0
SalesYTD                0
SalesLastYear           0
Is_Weekend              0
Month_Sin               0
Month_Cos               0
Day_Sin                 0
Day_Cos                 0
Category_Accessories    0
Category_Bikes          0
Category_Clothing       0
Category_Components     0
OrderID                 0
dtype: int64

In [80]:
transformed_df.head()

,OrderQty,UnitPrice,LineTotal,TotalDue,ListPrice,StandardCost,Year,TerritoryID,SalesYTD,SalesLastYear,Is_Weekend,Month_Sin,Month_Cos,Day_Sin,Day_Cos,Category_Accessories,Category_Bikes,Category_Clothing,Category_Components,OrderID
0,1,3578.2700,3578.2700,3953.9884,3578.27,2171.2942,2011,6,6.771829e+06,5.693989e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43697
1,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,7,4.772398e+06,2.396540e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43698
2,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,1,7.887187e+06,3.298694e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43699
3,1,699.0982,699.0982,772.5036,782.99,486.7066,2011,4,1.051085e+07,5.366576e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43700
4,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,9,5.977815e+06,2.278549e+06,0,0.5,0.707107,-2.449294e-16,1.0,0.0,1.0,0.0,0.0,43701


In [49]:
transformed_df.isnull().sum()

SalesOrderID            0
OrderQty                0
UnitPrice               0
LineTotal               0
TotalDue                0
ListPrice               0
StandardCost            0
Year                    0
TerritoryID             0
SalesYTD                0
SalesLastYear           0
Is_Weekend              0
Month_Sin               0
Month_Cos               0
Day_Sin                 0
Day_Cos                 0
Category_Accessories    0
Category_Bikes          0
Category_Clothing       0
Category_Components     0
dtype: int64

In [82]:
transformed_df['OrderID'].unique().size

31456

In [83]:
transformed_df['OrderID'].value_counts()

OrderID
51721    67
53465    67
51739    67
51120    66
51160    66
         ..
50838     1
50837     1
68152     1
50836     1
43697     1
Name: count, Length: 31456, dtype: int64

In [85]:
transformed_df[transformed_df['OrderID'] == 51721]

,OrderQty,UnitPrice,LineTotal,TotalDue,ListPrice,StandardCost,Year,TerritoryID,SalesYTD,SalesLastYear,Is_Weekend,Month_Sin,Month_Cos,Day_Sin,Day_Cos,Category_Accessories,Category_Bikes,Category_Clothing,Category_Components,OrderID
37354,12,31.8942,375.0758,109948.7407,54.99,20.5663,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,1.0,0.0,0.0,0.0,51721
37355,6,24.2940,145.7640,109948.7407,40.49,17.9776,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,0.0,1.0,51721
37357,3,29.9940,89.9820,109948.7407,49.99,38.4923,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,1.0,0.0,51721
37358,1,149.8740,149.8740,109948.7407,249.79,136.7850,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,0.0,1.0,51721
37359,3,16.2720,48.8160,109948.7407,27.12,12.0413,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,0.0,1.0,51721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37420,2,323.9940,647.9880,109948.7407,539.99,294.5797,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,1.0,0.0,0.0,51721
37422,8,29.9940,239.9520,109948.7407,49.99,38.4923,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,1.0,0.0,51721
37423,2,31.5840,63.1680,109948.7407,52.64,23.3722,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,0.0,1.0,51721
37424,4,158.4300,633.7200,109948.7407,264.05,144.5938,2013,1,7.887187e+06,3.298694e+06,1,1.224647e-16,1.0,-0.201299,1.096038,0.0,0.0,0.0,1.0,51721


In [86]:
test_df = transformed_df.drop(['OrderID'], axis=1)
test_df.duplicated().sum()

12048

In [ ]:
order_features = transformed_df.groupby('OrderID').agg(
    
)

order_features.head()

,OrderQty,UnitPrice,LineTotal,TotalDue,ListPrice,StandardCost,Year,TerritoryID,SalesYTD,SalesLastYear,Is_Weekend,Month_Sin,Month_Cos,Day_Sin,Day_Cos,Category_Accessories,Category_Bikes,Category_Clothing,Category_Components,OrderID
0,1,3578.2700,3578.2700,3953.9884,3578.27,2171.2942,2011,6,6.771829e+06,5.693989e+06,0,5.000000e-01,0.707107,-2.449294e-16,1.000000,0.0,1.0,0.0,0.0,43697
1,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,7,4.772398e+06,2.396540e+06,0,5.000000e-01,0.707107,-2.449294e-16,1.000000,0.0,1.0,0.0,0.0,43698
2,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,1,7.887187e+06,3.298694e+06,0,5.000000e-01,0.707107,-2.449294e-16,1.000000,0.0,1.0,0.0,0.0,43699
3,1,699.0982,699.0982,772.5036,782.99,486.7066,2011,4,1.051085e+07,5.366576e+06,0,5.000000e-01,0.707107,-2.449294e-16,1.000000,0.0,1.0,0.0,0.0,43700
4,1,3399.9900,3399.9900,3756.9890,3399.99,1912.1544,2011,9,5.977815e+06,2.278549e+06,0,5.000000e-01,0.707107,-2.449294e-16,1.000000,0.0,1.0,0.0,0.0,43701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121237,1,21.9800,21.9800,34.2219,21.98,8.2205,2014,6,6.771829e+06,5.693989e+06,0,1.224647e-16,1.000000,-2.012985e-01,1.096038,1.0,0.0,0.0,0.0,75122
121238,1,8.9900,8.9900,34.2219,8.99,6.9223,2014,6,6.771829e+06,5.693989e+06,0,1.224647e-16,1.000000,-2.012985e-01,1.096038,0.0,0.0,1.0,0.0,75122
121239,1,21.9800,21.9800,209.9169,21.98,8.2205,2014,6,6.771829e+06,5.693989e+06,0,1.224647e-16,1.000000,-2.012985e-01,1.096038,1.0,0.0,0.0,0.0,75123
121240,1,159.0000,159.0000,209.9169,159.00,59.4660,2014,6,6.771829e+06,5.693989e+06,0,1.224647e-16,1.000000,-2.012985e-01,1.096038,1.0,0.0,0.0,0.0,75123


In [78]:
order_features.duplicated().sum()

AttributeError: 'DataFrameGroupBy' object has no attribute 'duplicated'